In [1]:
# Import Packages
import os
import sys
import random
import time

import pandas as pd
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import matplotlib
import matplotlib.pyplot as plt
import json
from web3 import Web3
import requests
import pickle

%matplotlib inline

# Fetching pool data

In [2]:
# Creating filename for pool data file
filename = 'data_sources/' + 'Uniswapv2' + '.csv'

# Initiating subgraph client
sample_transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
    verify=True,
    retries=5)

client = Client(transport=sample_transport)

In [3]:
''' 
Function to fetch pool data sorted b volumeUSD in descending order
@param offset no. of entries to skip
@retun str Pool data

data obtained will be in format 
{
  "data": {
    "pairs": [
      {
        "id": "",
        "token0": {
          "symbol": ""
        },
        "token1": {
          "symbol": ""
        },
        "volumeUSD": ""
      }
}
'''
def fetch_query(offset=0) -> str :
    # Query to fetch pool data sorted by volumeUSD in descending order
    query = '''                
                {
                    pairs(first:1000, offset:, orderBy:volumeUSD, orderDirection: desc)
                    {
                        id
                        token0
                        {
                            symbol
                        }
                        token1
                        {
                            symbol
                        }
                        volumeUSD
                    }
                }
            '''
    # Initiating param offset 
    offset = 'offset:' + str(offset) 
    query = query.replace('offset:', offset)

    # return query with pool data
    return query

In [4]:
# Code block to check funtioning of fetch_query()
# response = client.execute(gql(fetch_query(0)))
    
# print(type(response))
# print(type(response))
# print(response.keys())

In [5]:
# Object to store the pairs data
pools = list()

In [6]:
'''
This will fetch pool data from subgraph of top pools sorted by volume in descending order
'''

i = 0
while(i<=20000):
    # Get query for the next 1000 pools
    response = client.execute(gql(fetch_query(i)))
    
    # accesing pairs data
    response = response['pairs']
    
    # res is a single pair data
    for res in response:
        # appending signle pair data to master data
        pools.append(res)
            
            
    i += 1000

In [7]:
# No of entries in pools
print(len(pools))
# print(pools)
# print(pools[0])

21000


In [8]:
# Creating dataframe object from pools list
df = pd.DataFrame(pools)

In [9]:
df.head()

,id,token0,token1,volumeUSD
0,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,{'symbol': 'USDC'},{'symbol': 'WETH'},36444362873.91855316049964026422881
1,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,{'symbol': 'WETH'},{'symbol': 'USDT'},35317160704.72943336475104633328177
2,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,{'symbol': 'DAI'},{'symbol': 'WETH'},15222708986.01430049817276261030296
3,0x23fe4ee3bd9bfd1152993a7954298bb4d426698f,{'symbol': 'SCAMMY'},{'symbol': 'WETH'},11029715648.72159798743878461135205
4,0xe5ffe183ae47f1a0e4194618d34c5b05b98953a8,{'symbol': 'welp'},{'symbol': 'WETH'},11017636252.92521755536995878068531


In [10]:
# This will rename the params of dictionary in token0 column
def token0_rename(x):
    x = dict(x)
    # x['token0_name']= x.pop('name')
    x['token0_symbol']= x.pop('symbol')
    return x


# This will rename the params of dictionary in token1 column
def token1_rename(x):
    x = dict(x)
    # x['token1_name']= x.pop('name')
    x['token1_symbol']= x.pop('symbol')
    return x

# Renaming keys of dictionary in token0 and token1 columns
df  = pd.concat([df['id'], df['token0'].apply(token0_rename), df['token1'].apply(token1_rename), df['volumeUSD']], axis=1)
df.head()

,id,token0,token1,volumeUSD
0,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,{'token0_symbol': 'USDC'},{'token1_symbol': 'WETH'},36444362873.91855316049964026422881
1,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,{'token0_symbol': 'WETH'},{'token1_symbol': 'USDT'},35317160704.72943336475104633328177
2,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,{'token0_symbol': 'DAI'},{'token1_symbol': 'WETH'},15222708986.01430049817276261030296
3,0x23fe4ee3bd9bfd1152993a7954298bb4d426698f,{'token0_symbol': 'SCAMMY'},{'token1_symbol': 'WETH'},11029715648.72159798743878461135205
4,0xe5ffe183ae47f1a0e4194618d34c5b05b98953a8,{'token0_symbol': 'welp'},{'token1_symbol': 'WETH'},11017636252.92521755536995878068531


In [11]:
df.shape

(21000, 4)

In [12]:
# This will format the token0 and token1 columns
df  = pd.concat([df['id'], df['token0'].apply(pd.Series), df['token1'].apply(pd.Series), df['volumeUSD']], axis=1)

In [13]:
df.head(10)

,id,token0_symbol,token1_symbol,volumeUSD
0,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,USDC,WETH,36444362873.91855316049964026422881
1,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,WETH,USDT,35317160704.72943336475104633328177
2,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,DAI,WETH,15222708986.01430049817276261030296
3,0x23fe4ee3bd9bfd1152993a7954298bb4d426698f,SCAMMY,WETH,11029715648.72159798743878461135205
4,0xe5ffe183ae47f1a0e4194618d34c5b05b98953a8,welp,WETH,11017636252.92521755536995878068531
5,0xf9c1fa7d41bf44ade1dd08d37cc68f67ae75bf92,WETH,Ian Laphan fan token,10960535614.16002391682295420606054
6,0x382a9a8927f97f7489af3f0c202b23ed1eb772b5,generalize fix for rebass tokens,WETH,10931540454.240024434064419683457
7,0xbb2b8038a1640196fbe3e38816f3e67cba72d940,WBTC,WETH,8732529527.91165971630102719767514
8,0x811beed0119b4afce20d2583eb608c6f7af1954f,SHIB,WETH,8731956171.781565487788939993225987
9,0x94b0a3d511b6ecdb17ebf877278ab030acb0a878,FEI,WETH,7748431296.581047398325894302931402


In [14]:
# Adding platform and API column in dataframe
df['Platform'] = 'UniswapV2'
# df['API'] = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2/'

In [15]:
print(df.shape)
df.head()

(21000, 5)


,id,token0_symbol,token1_symbol,volumeUSD,Platform
0,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,USDC,WETH,36444362873.91855316049964026422881,UniswapV2
1,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,WETH,USDT,35317160704.72943336475104633328177,UniswapV2
2,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,DAI,WETH,15222708986.01430049817276261030296,UniswapV2
3,0x23fe4ee3bd9bfd1152993a7954298bb4d426698f,SCAMMY,WETH,11029715648.72159798743878461135205,UniswapV2
4,0xe5ffe183ae47f1a0e4194618d34c5b05b98953a8,welp,WETH,11017636252.92521755536995878068531,UniswapV2


In [16]:
# Drop duplicates from the dataset

df = df.drop_duplicates()
print(df.shape)
df.head()

(1018, 5)


,id,token0_symbol,token1_symbol,volumeUSD,Platform
0,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,USDC,WETH,36444362873.91855316049964026422881,UniswapV2
1,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,WETH,USDT,35317160704.72943336475104633328177,UniswapV2
2,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,DAI,WETH,15222708986.01430049817276261030296,UniswapV2
3,0x23fe4ee3bd9bfd1152993a7954298bb4d426698f,SCAMMY,WETH,11029715648.72159798743878461135205,UniswapV2
4,0xe5ffe183ae47f1a0e4194618d34c5b05b98953a8,welp,WETH,11017636252.92521755536995878068531,UniswapV2


In [17]:
'''
Saving dataframe to local .csv files

if same name file already exists, append the data
else save the data to a new file
'''

# if os.path.exists(filename):
#     print('File already exists')
#     df1 = pd.read_csv(filename)
#     df1 = pd.concat([df1,df])
#     df1.to_csv(filename, index=False)
#     print('Data appended')
# else:
#     df.to_csv(filename, index=False)
#     print('Data Saved')

'\nSaving dataframe to local .csv files\n\nif same name file already exists, append the data\nelse save the data to a new file\n'

# Sorting wallet data

In [18]:
# Infura Details
PROJECT_ID = "1b0fc3af1d194e8ea8db248ccf926085"


# Etherscan API Details
ETHERSCAN_API_KEY = "P4XI3P1XKMUFPGIKGDNEJ8RAV6TBE317MW"
ETH_MAINNET = "https://api.etherscan.io/"

In [19]:
# Web3 Handle
w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/"+str(PROJECT_ID)))

In [20]:
# Fetch ABI from contract address
## Needs Contract Address and API Key as an input

def etherscan_abi_request(_contractAddress, _apiKey):
    '''
        Etherscan ABI Request Method
        Args::
            _contractAddress: Ethereum Address of the contract
            _apiKey: Etherscan API Key
        Returns::
            ethResJS: Etherscan ABI request result in JSON format
    '''
    ETHERSCAN_ABI_REQUEST = "https://api.etherscan.io/api?module=contract&action=getabi&address={0}&apikey={1}".format(_contractAddress, _apiKey)
    ethResJS = (requests.get(ETHERSCAN_ABI_REQUEST)).json()['result']
#     ethRes = (requests.get(ETHERSCAN_ABI_REQUEST))
    return ethResJS

In [21]:
# Fetch transactions history from contract address
## Needs Contract Address and API Key as an input

def etherscan_transaction_history(_contractAddress, _apiKey):
    '''
    
        Etherscan Transaction History Method
        Args::
            _contractAddress: Ethereum Address of the contract
            _apiKey: Etherscan API Key
        Returns::
            ethResJS: Etherscan ABI request result in JSON format
    '''
#     ETHERSCAN_ABI_REQUEST = ETHERSCAN_TRANSACTION_HISTORY = "https://api.etherscan.io/api?module=account&action=txlist&address={0}&startblock=0&endblock=99999999&page=1&offset=100&sort=asc&apikey={1}".format(_contractAddress, _apiKey)
    ETHERSCAN_TRANSACTION_HISTORY = "https://api.etherscan.io/api?module=account&action=txlist&address={0}&sort=asc&apikey={1}".format(_contractAddress, _apiKey)
    ethResJS = (requests.get(ETHERSCAN_TRANSACTION_HISTORY)).json()['result']
#     ethRes = (requests.get(ETHERSCAN_ABI_REQUEST))
    return ethResJS

In [22]:
# Fetch transactions details from transaction hash
## Needs transaction hash and API Key as an input

def etherscan_get_transaction_receipt(_transactionHash, _apiKey):
    '''
        Etherscan Get Transaction Receipt Method
        Args::
            _transactionHash: Transaction hash of the transaction
            _apiKey: Etherscan API Key
        Returns::
            etherResJS: Etherscan Transaction Receipt in JSON Format
    '''
    ETHERSCAN_TRANSACTION_RECEIPT = "https://api.etherscan.io/api?module=proxy&action=eth_getTransactionReceipt&txhash={0}&apikey={1}".format(_transactionHash, _apiKey)
#     ETHERSCAN_TRANSACTION_HISTORY = "https://api.etherscan.io/api?module=account&action=txlist&address={0}&sort=asc&apikey={1}".format(_contractAddress, _apiKey)
    ethResJS = (requests.get(ETHERSCAN_TRANSACTION_RECEIPT)).json()['result']
#     ethRes = (requests.get(ETHERSCAN_ABI_REQUEST))
    return ethResJS
    
    

In [23]:
# Sample Uniswap V2 Pool Address
# USDC-WETH Pair
UNI_V2_TEST_ADDRESS = "0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc"

In [24]:
# Fetching ABI
test_res_v2 = etherscan_abi_request(UNI_V2_TEST_ADDRESS, ETHERSCAN_API_KEY )

In [25]:
test_res_v2

'[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"Burn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"}],"name":"

In [26]:
# Converting to checksum address for Uniswap v2
UNI_V2_TEST_ADDRESS_CHECKSUM = w3.toChecksumAddress(UNI_V2_TEST_ADDRESS)

In [27]:
# Initiating contract instance to interact with web3
UNI_V2_TEST_CONTRACT_INSTANCE = w3.eth.contract(address=UNI_V2_TEST_ADDRESS_CHECKSUM, abi=test_res_v2)

In [28]:
# Getting the Token0 and Token1 Addreess
UNI_V2_TOKEN0_ADDRESS = UNI_V2_TEST_CONTRACT_INSTANCE.functions.token0().call()
UNI_V2_TOKEN1_ADDRESS = UNI_V2_TEST_CONTRACT_INSTANCE.functions.token1().call()

In [29]:
# Fetching transactions history
## Limit : 10000, set by etherscan
test_res_eth_usdt = etherscan_transaction_history(UNI_V2_TEST_ADDRESS, ETHERSCAN_API_KEY)

In [30]:
test_res_eth_usdt

[{'blockNumber': '10019997',
  'timeStamp': '1588866390',
  'hash': '0x71d6574a2d743cafc42e12bd1996f18c28d6231e7bfc8268b8133f71eb82d2a4',
  'nonce': '908',
  'blockHash': '0x9f588bd2dc2c574013c292921fbfabdd354f799d59e753e435d0f8397f014d66',
  'transactionIndex': '33',
  'from': '0x66666600e43c6d9e1a249d29d58639dedfcd9ade',
  'to': '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc',
  'value': '0',
  'gas': '66429',
  'gasPrice': '24200000000',
  'isError': '0',
  'txreceipt_status': '1',
  'input': '0x095ea7b30000000000000000000000003f6cdd93e4a1c2df9934cb90d09040ccfc155f93ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff',
  'contractAddress': '',
  'cumulativeGasUsed': '1701273',
  'gasUsed': '44286',
  'confirmations': '3978801'},
 {'blockNumber': '10022979',
  'timeStamp': '1588905808',
  'hash': '0xb63b01b54edeb4b5663bfdedce136c958bb8a53c9947130762e7d9161a7a6c5b',
  'nonce': '918',
  'blockHash': '0xb758d71db6b9ea80b85c8974676ae3aa9ae0658aab052c527f49b494803eb386',
  'tran

In [31]:
len(test_res_eth_usdt)

10000

In [32]:
test_res_eth_usdt[0]

{'blockNumber': '10019997',
 'timeStamp': '1588866390',
 'hash': '0x71d6574a2d743cafc42e12bd1996f18c28d6231e7bfc8268b8133f71eb82d2a4',
 'nonce': '908',
 'blockHash': '0x9f588bd2dc2c574013c292921fbfabdd354f799d59e753e435d0f8397f014d66',
 'transactionIndex': '33',
 'from': '0x66666600e43c6d9e1a249d29d58639dedfcd9ade',
 'to': '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc',
 'value': '0',
 'gas': '66429',
 'gasPrice': '24200000000',
 'isError': '0',
 'txreceipt_status': '1',
 'input': '0x095ea7b30000000000000000000000003f6cdd93e4a1c2df9934cb90d09040ccfc155f93ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff',
 'contractAddress': '',
 'cumulativeGasUsed': '1701273',
 'gasUsed': '44286',
 'confirmations': '3978801'}

In [33]:
# This is a topic dictionary to know the hex values of different topics, e.g. Transfer, Swap
UNISWAP_V2_TOPIC_DICTIONARY = {
    "0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9" : "PairCreated",
    "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef" : "Transfer",
    "0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1" : "Sync",
    "0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822" : "Swap",
    "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925" : "Approval",
    "0xdccd412f0b1252819cb1fd330b93224ca42612892bb3f4f789976e6d81936496": "Burn",
    "0xc12b12caff7d3a4e7e96205e010dce0f62a6e94ad6c798378a375566da377f0b" : "Liquidate",
    "0x0" : "Mint",
}

In [34]:
# Initiating dictionaries to store the data fetched by using wallet address as key

## Dictionary for transactions data by wallet address
"""
format-
"address": [List of transactions]
"""
MASTER_TRANSACTIONS = {
    "address1": []
}


## Dictionary for logs data by wallet address
"""
format-
"address": [List of logs in format[topic, address_interacted_with]]
"""
MASTER_LOGS = {
    "address1": {"TRANSFERS": {}, "SWAPS":{}}
}


In [35]:
# Keys of UNISWAP_V2_TOPIC_DICTIONARY
keys = UNISWAP_V2_TOPIC_DICTIONARY.keys()

In [ ]:
# Looping through the transactions of a pool

for transaction in test_res_eth_usdt:
    # Fetching transaction hash
    tx_hash = transaction['hash']

    # Initiating the loop inside try block to prevent operating halting in the middle
    try:
        # Fetching transaction receipt
        tx_receipt = etherscan_get_transaction_receipt(tx_hash,ETHERSCAN_API_KEY )
        # Fetch `from`(wallet address) from transaction receipt
        _from = tx_receipt['from']

        # Appending transaction hash to wallet address in MASTER_TRANSACTIONS
        MASTER_TRANSACTIONS.setdefault(_from, []).append(tx_hash)
        
        # Fetching logs of a transaction from transaction receipt
        logs = tx_receipt['logs']
        # Looping though the logs of each transaction
        for log in logs:
            # Address interacted with
            address = log['address']
            topic = log['topics'][0]

            # Converting from topic hex to event name from UNISWAP_V2_TOPIC_DICTIONARY
            if topic in keys:
                topic = UNISWAP_V2_TOPIC_DICTIONARY[topic]
            else:
                topic = topic

            # Appending logs from transaction to wallet address in MASTER_LOGS
            MASTER_LOGS.setdefault(_from, []).append([topic, address])
        
    # Catching Error
    except Exception as err:
        # Transaction hash in which error is caught
        print("Transaction hash: " + str(tx_hash))
        # Error
        print(err)
        pass


Transaction hash: 0x1432be0cc7f1eca794ec3c35e496a420217dc383c7409247173a05f375036e8c
Expecting value: line 1 column 1 (char 0)
Transaction hash: 0xfd68432d68526d28afeeb056d211d8ecb210df10f963c7070ce63907b9899c3f
Expecting value: line 1 column 1 (char 0)
Transaction hash: 0xa2eab027d442225cfd31382053dcde5a76751a517c6d43517a8529b50bac0147
Expecting value: line 1 column 1 (char 0)
Transaction hash: 0xc999ea4bf87824985378e667c9cfc62607dbede0cd5d211e1ce0b51f5ee307de
Expecting value: line 1 column 1 (char 0)
Transaction hash: 0x73a336b391cbbc17e91d64126bd86702f58d4a5a9ff077d098688db3943e6919
Expecting value: line 1 column 1 (char 0)
Transaction hash: 0x08626916a58acf8ad7fd03008258ffe5f3fc40b5e3fdfc157b2f885c605b8399
Expecting value: line 1 column 1 (char 0)
Transaction hash: 0xd8df9ce8bb6ee01184e46532b901ddb25e34251f8efc9b452f4459f67ce9f465
Expecting value: line 1 column 1 (char 0)
Transaction hash: 0x216dc38702b5bbba5ea72e5ee793a1c0653f803d2a1f53907d40bad926bfa930
Expecting value: line 1 co

In [ ]:
# Saving MASTER_TRANSACTIONS as pickle file
with open('./pickle_files/MASTER_TRANSACTION.pickle', 'wb') as file:
    pickle.dump(MASTER_TRANSACTIONS, file, protocol=pickle.HIGHEST_PROTOCOL)

# Saving MASTER_LOGS as pickle file
with open('./pickle_files/MASTER_LOGS.pickle', 'wb') as file:
    pickle.dump(MASTER_LOGS, file, protocol=pickle.HIGHEST_PROTOCOL)

## Looping through all the pairs of UniswapV2 for fetching transaction details


In [ ]:
## Initiating pools dataframe
pools_df = pd.read_csv('./data_sources/Uniswapv2.csv')
pools_df.head()

In [ ]:
# Fetching pools' address as a set
pools = set(pools_df['id'])
print(len(pools))
pools

In [ ]:
# Viewing Wallet Profiling Samples - Transactions
MASTER_TRANSACTIONS

In [ ]:
# Viewing Wallet Profiling Samples - Logs
MASTER_LOGS

In [ ]:
with open("MASTER_TRANSACTIONS.json", "w") as outfile:
    json.dump(MASTER_TRANSACTIONS, outfile)

In [ ]:
with open("MASTER_LOGS.json", "w") as outfile:
    json.dump(MASTER_LOGS, outfile)

In [ ]:
max_add = "address1"
for sample in MASTER_TRANSACTIONS.keys():
    print(sample)
    if len(MASTER_TRANSACTIONS[sample]) > len(MASTER_TRANSACTIONS[max_add]):
        print("Address Changed")
        max_add = sample

In [ ]:
# VIEW LOG FOR SAMPLE ADDRESSES

In [ ]:
MASTER_LOGS[max_add]

In [ ]:
MASTER_TRANSACTIONS[max_add]